In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Directories for training and test data
train_dir = '../ML2Project/Dataset_Original/train'
test_dir = '../ML2Project/Dataset_Original/test'

# Image sizes and batch size
IMAGE_SIZE = (224, 224)  
BATCH_SIZE = 32


# Lade die Bilder aus den Verzeichnissen
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999,
    validation_split=0.25,
    subset="training"
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999,
    validation_split=0.25,
    subset="validation"
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=999
)

# Normierungsschicht hinzufügen
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Normiere die Datensätze
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetching und Caching für Effizienz
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Definiere die Datenaugmentierungsschichten
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

# Definiere das CNN-Modell
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),  # Eingabeform hinzufügen
    data_augmentation,  # Datenaugmentierungsschichten hinzufügen
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)  # 3 Klassen
])

# Kompiliere das Modell
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Trainiere das Modell
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)

# Evaluieren und visualisieren
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
import sys
sys.path.append('dataPreparation.py') # Import the dataPreparation.py script
from dataPreparation import load_and_prepare_datasets, get_data_augmentation

# Directories for training and test data
train_dir = '../ML2Project/Dataset_Original/train'
test_dir = '../ML2Project/Dataset_Original/test'

# Image sizes and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Lade und bereite die Datensätze vor
train_dataset, validation_dataset, test_dataset = load_and_prepare_datasets(train_dir, test_dir, IMAGE_SIZE, BATCH_SIZE)

# Hole die Datenaugmentierungsschichten
data_augmentation = get_data_augmentation()

# Definiere das CNN-Modell
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),  # Eingabeform hinzufügen
    data_augmentation,  # Datenaugmentierungsschichten hinzufügen
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)  # 3 Klassen
])

# Kompiliere das Modell
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Trainiere das Modell
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)

# Evaluieren und visualisieren
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')

import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()